In [3]:
# load packages
import os
import glob
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import backend
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from tqdm import tqdm

In [4]:
# path of model checkpoints
checkpoint_path = '../../data/models/20200928_nvidia_ai/'
extracted_coefficients_directory = '../../data/models/extracted_coefficients/20200928_nvidia_ai/'
shard_path = '../../data/commaai/train_files_from_bag_test'

In [5]:
shard_files = glob.glob(os.path.join(shard_path, "*.tfrecords")) 

## Get $\beta$
0. Build model

In [ ]:
# build model
Input = tf.keras.layers.Input(shape=(66, 200, 3,), name='image')
x = Conv2D(24, kernel_size=(5, 5), activation='relu', strides=(2, 2))(Input)
x = BatchNormalization()(x)
x = Conv2D(36, kernel_size=(5, 5), activation='relu', strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(48, kernel_size=(5, 5), activation='relu', strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
#x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(1164)(x)
x = Dropout(0.5)(x)
x = Dense(100)(x)
x = Dropout(0.2)(x)
x = Dense(50)(x) 
x = Dense(10)(x)
Output = Dense(1, name = 'output_layer')(x)

keras_model = tf.keras.models.Model(
      inputs = [Input], outputs = [Output])

def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

keras_model.load_weights(tf.train.latest_checkpoint(checkpoint_path))

adam_optimizer = keras.optimizers.Adam(learning_rate=0.0005)
# old
#keras.optimizers.Adadelta(learning_rate = 0.001, rho = 0.95, epsilon = 1e-07, name = 'Adadelta')
keras_model.compile(
    loss = rmse,
    optimizer = adam_optimizer,
    metrics=[rmse, 'mse', 'mae'])

keras_estimator = tf.keras.estimator.model_to_estimator(
      keras_model=keras_model)
# to inspect weights:
#i = 0 
#for layer in keras_model.layers: 
#    i += 1
#    if i == 16:
#        print(layer.get_config(), layer.get_weights())

# get list of all variables in checkpoint file
# tf.train.list_variables(tf.train.latest_checkpoint(checkpoint_path))

In [ ]:
i = 0
for layer in keras_model.layers: 
    i += 1
    if i == 18:
        beta = layer.get_weights()
        print(layer.get_config()) #, layer.get_weights()

In [ ]:
beta

In [ ]:
# last element of beta is bias, exclude this 
# since copulas are location free -> beta_0 = 0
beta_coeff = beta[0]
np.savetxt(str(extracted_coefficients_directory +"beta.csv"), beta_coeff, delimiter=",")

## Get model for $B_{\zeta}$

In [ ]:
Input = tf.keras.layers.Input(shape=(66, 200, 3,), name='image')
x = Conv2D(24, kernel_size=(5, 5), activation='relu', strides=(2, 2))(Input)
x = BatchNormalization()(x)
x = Conv2D(36, kernel_size=(5, 5), activation='relu', strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(48, kernel_size=(5, 5), activation='relu', strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
#x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(1164)(x)
x = Dropout(0.5)(x)
x = Dense(100)(x)
x = Dropout(0.2)(x)
x = Dense(50)(x) 
Output = Dense(10)(x)

B_zeta = tf.keras.models.Model(
      inputs = [Input], outputs = [Output])

B_zeta.load_weights(tf.train.latest_checkpoint(checkpoint_path))

def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

adam_optimizer = keras.optimizers.Adam(learning_rate=0.0005)
# old
#keras.optimizers.Adadelta(learning_rate = 0.001, rho = 0.95, epsilon = 1e-07, name = 'Adadelta')
B_zeta.compile(
    loss = rmse,
    optimizer = adam_optimizer,
    metrics=[rmse, 'mse', 'mae'])

#B_zeta_estimator = tf.keras.estimator.model_to_estimator(
#      keras_model=B_zeta)

get training data

In [10]:
# maximum number of elements in shard is 240
bs = 240
def imgs_input_fn(filenames, perform_shuffle = False, repeat_count = 1, batch_size = bs): 
    
    # reads in single training example and returns it in a format that the estimator can
    # use
    def _parse_function(proto):
        # define your tfrecord again. Remember that you saved your image as a string.
        keys_to_features = {'image': tf.io.FixedLenFeature([], tf.string),
                            "label": tf.io.FixedLenFeature([], tf.float32),
                            'rows': tf.io.FixedLenFeature([], tf.int64),
                            'cols': tf.io.FixedLenFeature([], tf.int64),
                            'depth': tf.io.FixedLenFeature([], tf.int64),
                            "tr_label": tf.io.FixedLenFeature([], tf.float32),
                           }

        # Load one example
        parsed_example = tf.io.parse_single_example(proto, keys_to_features)

        image_shape = tf.stack([131 , 174, 3])
        image_raw = parsed_example['image']

        label = tf.cast(parsed_example['label'], tf.float32)
        tr_label = tf.cast(parsed_example['tr_label'], tf.float32)
        image = tf.io.decode_raw(image_raw, tf.uint8)
        image = tf.cast(image, tf.float32)

        image = tf.reshape(image, image_shape)
        image = tf.image.resize(image[33:99,37:137, :], [66,200])
        image = image/255
        
        return {'image':image}, tr_label, label
    
    dataset = tf.data.TFRecordDataset(filenames = filenames)
    dataset = dataset.map(_parse_function)
    if perform_shuffle:
        # Randomizes input using a window of 256 elements (read into memory)
        dataset = dataset.shuffle(buffer_size=256)
    dataset = dataset.repeat(repeat_count)  # Repeats dataset this # times
    dataset = dataset.batch(batch_size)  # Batch size to use
    iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
    batch_features, batch_tr_labels, batch_labels = iterator.get_next()
    
    return batch_features, batch_tr_labels, batch_labels

In [21]:
import pandas as pd
shardzero = shard_files[0]
images, tr_labels, labels = imgs_input_fn(shardzero)
tr_labels = np.array(tr_labels)
labels = np.array(labels)
df = pd.DataFrame({'tr_label': tr_labels, 'label': labels})

In [22]:
df

tr_label      label
0   -0.418915  -1.100000
1    2.202471  13.100000
2    1.191695 -23.600000
3    2.546087  23.600000
4    1.233022  32.799999
..        ...        ...
100 -2.695096 -28.700001
101  1.567552  33.299999
102 -2.752856 -33.299999
103      -inf -40.500000
104  0.935097  50.700001

[105 rows x 2 columns]

In [15]:
tr_labels in [-float('inf'), float('nan'), float('inf')]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
shardzero = shard_files[0]
images, labels = imgs_input_fn(shardzero)
images = np.array(images['image'])
labels = np.array(labels)
all_images = images
all_labels = labels
for shard in tqdm(shard_files[1:]):
    try:
        images, labels = imgs_input_fn(shard)
        images = np.array(images['image'])
        labels = np.array(labels)
        all_images = np.append(all_images, images, axis = 0)
        all_labels = np.append(all_labels, labels, axis = 0)
    except:
        pass

In [ ]:
B_zeta_train = B_zeta.predict(x = all_images)

In [ ]:
np.save(str(extracted_coefficients_directory + 'B_zeta_train.npy'), B_zeta_train)
np.save(str(extracted_coefficients_directory + 'labels_train.npy'), all_labels)

In [ ]:
pred = B_zeta_train.dot(beta[0]) + beta[1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(pred, all_labels, alpha = 0.2)

In [ ]:
all_labels